In [1]:
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END

In [2]:
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]

In [3]:
llm = ChatOllama(model="llama3.2:1b")

In [4]:
def process(state: AgentState) -> AgentState:
    """ 
    This node will solve the request you input
    """
    response = llm.invoke(state['messages'])

    state['messages'].append(AIMessage(content=response.content))
    print(f"\nAI Response: {response.content}")

    return state

In [5]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)

agent = graph.compile()

In [6]:
conversation_history = []

user_input = input("Enter: ")
print("Human Question: ", user_input)
while user_input != "exit":
    conversation_history.append(HumanMessage(content=user_input))

    result = agent.invoke({"messages": conversation_history})

    # print(result['messages'])
    conversation_history = result['messages']

    user_input = input("Enter:")
    print("Human Question: ", user_input)

# print(conversation_history)

Human Question:  Hi

AI Response: How can I help you today?
Human Question:  I am steve

AI Response: Nice to meet you, Steve. Is there something I can help you with or would you like to chat?
Human Question:  Who am i?

AI Response: Steve, isn't it? You're a person, and I'm not sure how we know each other, but I'll ask politely: Are you someone who's been talking to me, or is this the start of something new?
Human Question:  exit


In [7]:
with open("logging.txt", "w") as file:
    file.write("Your Conversation Log: \n")

    for message in conversation_history:
        if isinstance(message, HumanMessage):
            file.write(f"Human: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI   : {message.content}\n")
            file.write("\n")
    file.write("End of Conversation \n")

print("Conversation saved to logging.txt")

Conversation saved to logging.txt
